In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from get_method_here import get_method_here, def_model
import os
import cv2
import pickle
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Featuremap Visualisierung

In [ ]:
# Speicherverwaltung einstellen
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

def extract_from_end_until_underscore(input_string):
    last_underscore_index = input_string.rfind('_')
    if last_underscore_index != -1:
        result = input_string[last_underscore_index + 1:]
    else:
        result = input_string
    return result

# Lade das Modell
def load_model(model_name, model_path, device):
    _, _, arch, norm_type, _ = get_method_here(model_name, weights_path=model_path)
    model = def_model(arch, model_path, localize=False)
    model = model.to(device).eval()
    return model, norm_type

# Transformationen definieren
def get_transform(norm_type):
    from normalization import get_list_norm
    
    transform = []
    transform = transform + get_list_norm(norm_type)
    return transforms.Compose(transform)

# Benutzerdefinierte Colormap definieren
def get_custom_cmap():
    colors = [(0, 'green'), (0.5, 'white'), (1, 'red')]
    n_bins = 100  # Discretizes the interpolation into bins
    cmap_name = 'custom_cmap'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
    return cmap

# Ausgabekarte berechnen und visualisieren
def visualize_output_map(model, img_path, transform, device):
    image = Image.open(img_path).convert('RGB')
    input_img = transform(image).unsqueeze(0).to(device)

    # Modell ausführen
    output = model(input_img).squeeze().cpu().detach().numpy()
    print(output.shape)

    # Wertebereich für die Ausgabekarte festlegen
    vmin, vmax = -200, 200

    # Originalbild und Ausgabekarte visualisieren
    fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

    fig.text(0.5, 0.79, f'Modellausgabe für {extract_from_end_until_underscore(img_path)}', ha='center', fontsize=20)

    ax[0].imshow(image)
    ax[0].set_title('Originalbild', fontsize=16)

    im = ax[1].imshow(output, cmap=get_custom_cmap(), vmin=vmin, vmax=vmax, alpha=0.5)
    ax[1].set_title('Modellausgabe (Featuremap)', fontsize=16)

    # Farbleiste hinzufügen
    cbar = fig.colorbar(im, ax=ax.ravel().tolist(), shrink=0.5, aspect=10)
    cbar.set_label('Logits', fontsize=14)
    
    plt.savefig(f'../statistics/featuremaps/group-2/{os.path.basename(img_path)}', bbox_inches='tight')
    plt.show()

# Verwende CPU statt GPU, falls der Speicher auf der GPU nicht ausreicht
device = torch.device('cuda:0' if torch.cuda.is_available() and torch.cuda.memory_allocated() < 1e9 else 'cpu')
# Modell und Transformationen laden
model_latent, norm_type_latent = load_model('Grag2021_latent', 'weights/Grag2021_latent/model_epoch_best.pth', device)
transform_latent = get_transform(norm_type_latent)


In [ ]:
# Heatmapgenerierung und Abspeichern als Bilder
for img in os.listdir('../img-dataset/study/study-group-2/imgs'):
    visualize_output_map(model_latent, f'../img-dataset/study/study-group-2/imgs/{img}', transform_latent, device)

## Gesamtheatmap 3x3

In [ ]:
# 1 Heatmap in 9 Bereiche aufgeteilt
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

def extract_from_end_until_underscore(input_string):
    last_underscore_index = input_string.rfind('_')
    if last_underscore_index != -1:
        result = input_string[last_underscore_index + 1:]
    else:
        result = input_string
    return result

# Lade das Modell
def load_model(model_name, model_path, device):
    _, _, arch, norm_type, _ = get_method_here(model_name, weights_path=model_path)
    model = def_model(arch, model_path, localize=False)
    model = model.to(device).eval()
    return model, norm_type

# Transformationen definieren
def get_transform(norm_type):
    from normalization import get_list_norm
    
    transform = []
    transform = transform + get_list_norm(norm_type)
    return transforms.Compose(transform)

# Benutzerdefinierte Colormap definieren
def get_custom_cmap(vmin, vmax):
    abs_max = max(abs(vmin), abs(vmax))
    colors = [(0, 'green'), (0.5, 'white'), (1, 'red')]
    cmap_name = 'custom_cmap'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    return cmap, -abs_max, abs_max

# Ausgabekarte berechnen und visualisieren
def visualize_output_map(model, img_path, transform, device):
    image = Image.open(img_path).convert('RGB')
    input_img = transform(image).unsqueeze(0).to(device)

    # Modell ausführen
    output = model(input_img).squeeze().cpu().detach().numpy()
    print(output.shape)

    # Ursprüngliche Heatmap-Größe
    original_height, original_width = output.shape

    # Ziel-Rastergröße (3x3)
    target_height, target_width = 3, 3

    # Initialisiere die aggregierte Heatmap
    aggregated_output = np.zeros((target_height, target_width))

    # Aggregiere die Werte in die Ziel-Rasterbereiche
    for i in range(target_height):
        for j in range(target_width):
            # Bestimme die entsprechenden Bereiche in der ursprünglichen Heatmap
            start_y = i * original_height // target_height
            end_y = (i + 1) * original_height // target_height
            start_x = j * original_width // target_width
            end_x = (j + 1) * original_width // target_width

            # Summe der Werte in diesem Bereich
            aggregated_output[i, j] = np.sum(output[start_y:end_y, start_x:end_x])

    # Interpolation der aggregierten Werte auf die Bildgröße (480x640)
    interpolated_output = cv2.resize(aggregated_output, (640, 480), interpolation=cv2.INTER_NEAREST)

    # Wertebereich für die Ausgabekarte festlegen
    vmin, vmax = aggregated_output.min(), aggregated_output.max()
    cmap, adjusted_vmin, adjusted_vmax = get_custom_cmap(vmin, vmax)

    # Originalbild und interpolierte Ausgabekarte visualisieren
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.imshow(image)
    ax.set_title(extract_from_end_until_underscore(img_path))

    im = ax.imshow(interpolated_output, cmap=cmap, vmin=adjusted_vmin, vmax=adjusted_vmax, alpha=0.5)
    
    # Farbleiste hinzufügen
    cbar = fig.colorbar(im, ax=ax, shrink=0.5, aspect=10)
    cbar.set_label('Activation Intensity')
    
    plt.savefig(f'../statistics/3x3_heatmaps/group-2/{os.path.basename(img_path)}', bbox_inches='tight')
    plt.show()
    
# Verwende CPU statt GPU, falls der Speicher auf der GPU nicht ausreicht
device = torch.device('cuda:0' if torch.cuda.is_available() and torch.cuda.memory_allocated() < 1e9 else 'cpu')
# Modell und Transformationen laden
model_latent, norm_type_latent = load_model('Grag2021_latent', 'weights/Grag2021_latent/model_epoch_best.pth', device)
transform_latent = get_transform(norm_type_latent)
test_img_path = '../imgs/mixed/01_000000415727.jpg'

# Ausgabekarte visualisieren
#visualize_output_map(model_latent, test_img_path, transform_latent, device)

## Heatmaps nach Vorzeichen 3x3

In [ ]:
# 2 Heatmaps in 9 Bereiche
def extract_from_end_until_underscore(input_string):
    last_underscore_index = input_string.rfind('_')
    if last_underscore_index != -1:
        result = input_string[last_underscore_index + 1:]
    else:
        result = input_string
    return result

# Lade das Modell
def load_model(model_name, model_path, device):
    _, _, arch, norm_type, _ = get_method_here(model_name, weights_path=model_path)
    model = def_model(arch, model_path, localize=False)
    model = model.to(device).eval()
    return model, norm_type

# Transformationen definieren
def get_transform(norm_type):
    from normalization import get_list_norm
    
    transform = []
    transform = transform + get_list_norm(norm_type)
    return transforms.Compose(transform)

# Benutzerdefinierte Colormaps definieren
def get_positive_cmap():
    colors = [(0, 'white'), (1, 'red')]
    cmap_name = 'positive_cmap'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    return cmap

def get_negative_cmap():
    colors = [(0, 'green'), (1, 'white')]
    cmap_name = 'negative_cmap'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    return cmap

# Ausgabekarte berechnen und visualisieren
def visualize_output_maps(model, img_path, transform, device):
    image = Image.open(img_path).convert('RGB')
    input_img = transform(image).unsqueeze(0).to(device)

    # Modell ausführen
    output = model(input_img).squeeze().cpu().detach().numpy()
    print(output.shape)

    # Ursprüngliche Heatmap-Größe
    original_height, original_width = output.shape

    # Ziel-Rastergröße (3x3)
    target_height, target_width = 3, 3

    # Initialisiere die aggregierten Heatmaps
    aggregated_output_positive = np.zeros((target_height, target_width))
    aggregated_output_negative = np.zeros((target_height, target_width))

    # Aggregiere die Werte in die Ziel-Rasterbereiche
    for i in range(target_height):
        for j in range(target_width):
            # Bestimme die entsprechenden Bereiche in der ursprünglichen Heatmap
            start_y = i * original_height // target_height
            end_y = (i + 1) * original_height // target_height
            start_x = j * original_width // target_width
            end_x = (j + 1) * original_width // target_width

            # Summe der positiven Werte in diesem Bereich
            positive_values = output[start_y:end_y, start_x:end_x] * (output[start_y:end_y, start_x:end_x] > 0)
            aggregated_output_positive[i, j] = np.sum(positive_values)

            # Summe der negativen Werte in diesem Bereich
            negative_values = output[start_y:end_y, start_x:end_x] * (output[start_y:end_y, start_x:end_x] < 0)
            aggregated_output_negative[i, j] = np.sum(negative_values)

    # Interpolation der aggregierten Werte auf die Bildgröße (640x480)
    interpolated_output_positive = cv2.resize(aggregated_output_positive, (640, 480), interpolation=cv2.INTER_NEAREST)
    interpolated_output_negative = cv2.resize(aggregated_output_negative, (640, 480), interpolation=cv2.INTER_NEAREST)

    # Wertebereich für die Ausgabekarten festlegen
    abs_max_positive = np.max(aggregated_output_positive)
    abs_max_negative = abs(np.min(aggregated_output_negative))
    abs_max = max(abs_max_positive, abs_max_negative)

    positive_cmap = get_positive_cmap()
    negative_cmap = get_negative_cmap()

    # Originalbild und interpolierte Ausgabekarten visualisieren
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))

    # Set the title for the entire figure
    plt.figtext(0.5, 0.8, f'Aktivierung der Bildbereiche für {extract_from_end_until_underscore(img_path)}', ha='center', fontsize=18)

    axes[0].imshow(image)
    axes[0].set_title('"KI-generiert"', fontsize=14)
    im0 = axes[0].imshow(interpolated_output_positive, cmap=positive_cmap, vmin=0, vmax=abs_max, alpha=0.5)
    axes[0].set_xticks([640/6, 3*640/6, 5*640/6])
    axes[0].set_xticklabels(['A', 'B', 'C'])
    axes[0].set_yticks([480/6, 3*480/6, 5*480/6])
    axes[0].set_yticklabels(['1', '2', '3'])

    axes[1].imshow(image)
    axes[1].set_title('"echt"', fontsize=14)
    im1 = axes[1].imshow(interpolated_output_negative, cmap=negative_cmap, vmin=-abs_max, vmax=0, alpha=0.5)
    axes[1].set_xticks([640/6, 3*640/6, 5*640/6])
    axes[1].set_xticklabels(['A', 'B', 'C'])
    axes[1].set_yticks([480/6, 3*480/6, 5*480/6])
    axes[1].set_yticklabels(['1', '2', '3'])

    # Set tick parameters for both axes
    for ax in axes:
        ax.tick_params(axis='both', which='major', labelsize=12)

    # Farbleisten hinzufügen
    cbar0 = fig.colorbar(im0, ax=axes[0], shrink=0.5, aspect=10)
    cbar0.set_label('Positive Logits', fontsize=12)
    cbar0.ax.tick_params(labelsize=12)  # Set fontsize for colorbar ticks

    cbar1 = fig.colorbar(im1, ax=axes[1], shrink=0.5, aspect=10)
    cbar1.set_label('Negative Logits', fontsize=12)
    cbar1.ax.tick_params(labelsize=12)  # Set fontsize for colorbar ticks

    # Adjust layout to minimize whitespace
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the plot before showing, using bbox_inches='tight' to minimize whitespace
    save_path = f'../statistics/3x3_heatmaps/group-1/{os.path.basename(img_path)}'
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()


def get_heatmaps(model, img_path, transform, device):
    image = Image.open(img_path).convert('RGB')
    input_img = transform(image).unsqueeze(0).to(device)

    # Modell ausführen
    output = model(input_img).squeeze().cpu().detach().numpy()
    print(output.shape)

    # Ursprüngliche Heatmap-Größe
    original_height, original_width = output.shape

    # Ziel-Rastergröße (3x3)
    target_height, target_width = 3, 3

    # Initialisiere die aggregierten Heatmaps
    aggregated_output_positive = np.zeros((target_height, target_width))
    aggregated_output_negative = np.zeros((target_height, target_width))

    # Aggregiere die Werte in die Ziel-Rasterbereiche
    for i in range(target_height):
        for j in range(target_width):
            # Bestimme die entsprechenden Bereiche in der ursprünglichen Heatmap
            start_y = i * original_height // target_height
            end_y = (i + 1) * original_height // target_height
            start_x = j * original_width // target_width
            end_x = (j + 1) * original_width // target_width

            # Summe der positiven Werte in diesem Bereich
            positive_values = output[start_y:end_y, start_x:end_x] * (output[start_y:end_y, start_x:end_x] > 0)
            aggregated_output_positive[i, j] = np.sum(positive_values)

            # Summe der negativen Werte in diesem Bereich
            negative_values = output[start_y:end_y, start_x:end_x] * (output[start_y:end_y, start_x:end_x] < 0)
            aggregated_output_negative[i, j] = np.sum(negative_values)
    return (img_path, aggregated_output_positive, aggregated_output_negative)

# Verwende CPU statt GPU, falls der Speicher auf der GPU nicht ausreicht
device = torch.device('cuda:0' if torch.cuda.is_available() and torch.cuda.memory_allocated() < 1e9 else 'cpu')
# Modell und Transformationen laden
model_latent, norm_type_latent = load_model('Grag2021_latent', 'weights/Grag2021_latent/model_epoch_best.pth', device)
transform_latent = get_transform(norm_type_latent)

In [ ]:
visualize_output_maps(model_latent, f'../img-dataset/study/study-group-2/imgs/02_395343-short-sdxl.jpg', transform_latent, device)

In [3]:
# Heatmapgenerierung und Abspeichern als Bilder
for img in os.listdir('../img-dataset/study/study-group-1/imgs'):
    visualize_output_maps(model_latent, f'../img-dataset/study/study-group-1/imgs/{img}', transform_latent, device)

In [ ]:
# Numerische Daten der Heatmaps speichern
heatmaps_g1 = []
heatmaps_g2 = []
for img in os.listdir('../img-dataset/study/study-group-1/imgs'):
    heatmaps_g1.append(get_heatmaps(model_latent, f'../img-dataset/study/study-group-1/imgs/{img}', transform_latent, device))
with open('model_heatmaps_g1.pkl', 'wb') as f:
    pickle.dump(heatmaps_g1, f)

for img in os.listdir('../img-dataset/study/study-group-2/imgs'):
    heatmaps_g2.append(get_heatmaps(model_latent, f'../img-dataset/study/study-group-2/imgs/{img}', transform_latent, device))
with open('model_heatmaps_g2.pkl', 'wb') as f:
    pickle.dump(heatmaps_g2, f)

## Modellarchitektur

In [ ]:
import torch
from torchviz import make_dot
from get_method_here import get_method_here, def_model
import os

# Speicherverwaltung einstellen
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Lade das Modell
def load_model(model_name, model_path, device):
    _, _, arch, norm_type, _ = get_method_here(model_name, weights_path=model_path)
    model = def_model(arch, model_path, localize=False)
    model = model.to(device).eval()
    return model, norm_type

# Verwende CPU statt GPU, falls der Speicher auf der GPU nicht ausreicht
device = torch.device('cuda:0' if torch.cuda.is_available() and torch.cuda.memory_allocated() < 1e9 else 'cpu')

# Modell laden
model_latent, norm_type_latent = load_model('Grag2021_latent', 'weights/Grag2021_latent/model_epoch_best.pth', device)

# Modellarchitektur ausgeben
print("Modellarchitektur:")
print(model_latent)

# Testbild transformieren und durch das Modell laufen lassen
test_img_path = '../imgs/mixed/01_000000415727.jpg'
from PIL import Image
import torchvision.transforms as transforms

# Transformationen definieren
def get_transform(norm_type):
    from normalization import get_list_norm
    transform = []
    transform = transform + get_list_norm(norm_type)
    return transforms.Compose(transform)

transform_latent = get_transform(norm_type_latent)
image = Image.open(test_img_path).convert('RGB')
input_img = transform_latent(image).unsqueeze(0).to(device)

# Erzeuge den Graphen
output = model_latent(input_img)
dot = make_dot(output, params=dict(model_latent.named_parameters()))

# Speichere den Graphen als PDF oder PNG
dot.format = 'pdf'
dot.render('model_architecture')